In [1]:
import numpy as np
import pandas as pd
import os

#import sklearn.model_selection as model_selection
import tensorflow as tf
import keras
#import pydot
import matplotlib.pyplot as plt
import csv

In [2]:
BATCH_SIZE = 4
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
ResNet50_thresh = [
			0.74444923, 0.30997806, 0.4201866 , 0.22654252, 0.43615441,
			0.17688525, 0.10371951, 0.21240841, 0.01169568, 0.01127118,
			0.00631477, 0.1015508 , 0.09528982, 0.10901272, 0.97817575,
			0.0059347 , 0.05466259, 0.02932084, 0.06736068, 0.11088215,
			0.02574941, 0.24147759, 0.05154309, 0.29932558, 0.04224337,
			0.46476043, 0.02983662, 0.00598989
		  ]

In [4]:
Xception_thresh = [
            0.51997931, 0.97072391, 0.46045949, 0.99137787, 0.72512962,
            0.95618641, 0.97886023, 0.86301403, 0.99995389, 0.99964118,
            0.99946806, 0.98000044, 0.8887235 , 0.99623207, 0.98616731,
            0.96080434, 0.99745922, 0.99978051, 0.93036813, 0.84382995,
            0.99633127, 0.99982706, 0.99987868, 0.92638686, 0.99681511,
            0.91722482, 0.67764074, 0.99690612
            ]

In [5]:
HPAV100_thresh = [
            0.32236402, 0.05530944, 0.12785086, 0.05519807, 0.03973802,
            0.08109811, 0.07114721, 0.09052892, 0.00863046, 0.01262293,
            0.0132055 , 0.03374713, 0.02455985, 0.02844724, 0.04082179,
            0.00960584, 0.02783582, 0.01582063, 0.02419314, 0.08937495,
            0.02719273, 0.10485751, 0.02572011, 0.09735668, 0.01917352,
            0.26713884, 0.0207516 , 0.01519562
            ]

In [6]:
path_to_test = './test/'
_, _, filenames = next(os.walk(path_to_test))

for idx, file in enumerate(filenames):
    filenames[idx] = filenames[idx].split('_')[0]

test = pd.DataFrame(set(filenames), columns = ['Id'])
test['Path'] = path_to_test + test.Id
test['Predicted'] = ''
test

,Id,Path,Predicted
0,1f95b5ce-bad6-11e8-b2b9-ac1f6b6435d0,./test/1f95b5ce-bad6-11e8-b2b9-ac1f6b6435d0,
1,705d86e8-bacd-11e8-b2b8-ac1f6b6435d0,./test/705d86e8-bacd-11e8-b2b8-ac1f6b6435d0,
2,35208eb6-bad4-11e8-b2b8-ac1f6b6435d0,./test/35208eb6-bad4-11e8-b2b8-ac1f6b6435d0,
3,8e233686-bad3-11e8-b2b8-ac1f6b6435d0,./test/8e233686-bad3-11e8-b2b8-ac1f6b6435d0,
4,e53f7f52-bacd-11e8-b2b8-ac1f6b6435d0,./test/e53f7f52-bacd-11e8-b2b8-ac1f6b6435d0,
...,...,...,...
11697,2a15aece-bacb-11e8-b2b8-ac1f6b6435d0,./test/2a15aece-bacb-11e8-b2b8-ac1f6b6435d0,
11698,2b155dec-bac6-11e8-b2b7-ac1f6b6435d0,./test/2b155dec-bac6-11e8-b2b7-ac1f6b6435d0,
11699,210ce520-bad7-11e8-b2b9-ac1f6b6435d0,./test/210ce520-bad7-11e8-b2b9-ac1f6b6435d0,
11700,d1d3ea52-bad7-11e8-b2b9-ac1f6b6435d0,./test/d1d3ea52-bad7-11e8-b2b9-ac1f6b6435d0,


In [7]:
def load_data(path):
    red = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_red.png'), channels = 1), [2])
    blue = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_blue.png'), channels = 1), [2])
    green = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_green.png'), channels = 1), [2])
    img = tf.stack((red, green, blue), axis=2)
    return img

In [8]:
test_data = tf.data.Dataset.from_tensor_slices(np.array(test['Path']))
test_data = test_data.map(load_data, num_parallel_calls = AUTOTUNE)
test_data_batches = test_data.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)

In [9]:
test_data_batches

<PrefetchDataset shapes: (None, None, None, 3), types: tf.uint8>

In [10]:
def one_hot_decoder(mat):
    lis = []
    for i, arr in enumerate(mat):
        lis.append(' '.join(str(label) for label in [idx for idx, out in enumerate(arr) if out == 1]))
    return lis

def predict_csv(model, threshold, verbose = 1, test_path = './test/', save_path = './test.csv'):
    test_path = './test/'
    _, _, filenames = next(os.walk(test_path))

    for idx, file in enumerate(filenames):
        filenames[idx] = filenames[idx].split('_')[0]

    test = pd.DataFrame(set(filenames), columns = ['Id'])
    test['Path'] = test_path + test.Id
    test['Predicted'] = ''
    
    test_data = tf.data.Dataset.from_tensor_slices(np.array(test['Path']))
    test_data = test_data.map(load_data, num_parallel_calls = AUTOTUNE)
    test_data_batches = test_data.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)
    
    prediction = model.predict(test_data_batches, batch_size = BATCH_SIZE, verbose = verbose, steps = None,)
    for i in range(len(prediction)):
        prediction[i, prediction[i].argmax()] = 1
    prediction[prediction > threshold] = 1
    prediction[prediction <= threshold] = 0
    prediction = prediction.astype(int)
    
    test.Predicted = one_hot_decoder(prediction)
    test[['Id', 'Predicted']].to_csv(save_path, index = False, quoting=csv.QUOTE_NONE, quotechar="", escapechar =",")
    

In [11]:
# Load model_ResNet50
model_ResNet50 = keras.models.load_model('hpa_ResNet50_model.tf')
predict_csv(model_ResNet50, threshold = ResNet50_thresh, verbose = 1, save_path = './test_model_ResNet50.csv')

2926/2926 [==============================] - 148s 50ms/step


In [12]:
# Load model_Xception
model_Xception = keras.models.load_model('hpa_Xception_model.tf')
predict_csv(model_Xception, threshold = Xception_thresh, verbose = 1, save_path = './test_model_Xception.csv')

2926/2926 [==============================] - 158s 54ms/step


In [13]:
# Load model_HPAV100
#model_HPAV100 = keras.models.load_model('hpa_HPAV100_model.tf')
#predict_csv(model_HPAV100, threshold = HPAV100_thresh, verbose = 1, save_path = './test_model_HPAV100.csv')